In [70]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [71]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"WLDN"}, 
             {'symbol':"SSRM"}, 
             {'symbol':"LRN", 'basis':137.913}, 
             {'symbol':"UNFI", 'basis':22.200}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':60.671}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':68.738}, 
             {'symbol':"ALL", 'basis':195.419}, 
             {'symbol':"LC", 'basis':9.384}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':11.617}, 
             {'symbol':"AGX", 'basis':0.010}, 
             {'symbol':"POWL", 'basis':181.166}, 
             {'symbol':"PPC", 'basis':45.089}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':73.160}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':101.620}, 
             {'symbol':"TWLO", 'basis':87.178}, 
             {'symbol':"PEP"}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':225.525}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"WLDN"}, 
                   {'symbol':"SSRM"}, 
                   {'symbol':'BCS', 'basis':18.580}, # UK - Financials
                   {'symbol':'PUK'}, # Hong Kong - Financials
                   {'symbol':'FINV'}, # China - Financials
                   {'symbol':'PSIX'}, # U.S. - Industrials
                   {'symbol':'NGD'}, # Canada - Materials (Gold)
                   {'symbol':'GFI'}, # South Africa - Materials (Gold)
                   {'symbol':"CRDO", 'basis':17.173}, 
                   {'symbol':"COMM"}, 
                   {'symbol':'BKTI'}, # United States - Communication Equipment
                   {'symbol':"UNFI", 'basis':22.950}, 
                   {'symbol':"MFC"}, 
                   {'symbol':"EZPW", 'basis':17.715}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':68.586}, 
                   {'symbol':"ITRN"}, 
                   {'symbol':"PYPL", 'basis':69.126}, 
                   {'symbol':"LC"}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"CCL", 'basis':17.866}, 
                   {'symbol':"PPC", 'basis':0}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"BLBD"}, 
                   {'symbol':"GM"}, 
                   {'symbol':"OKTA", 'basis':104.950}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   #{'symbol':"GRBK"}, 
                   #{'symbol':"CAAP"},
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.19},
                   {'symbol':"QUBT"},
                   {'symbol':"RGTI"}
                   ]

In [72]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [73]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=.99, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [74]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,BCS,18.580,18.40,16.71,44.88,17.15,9.02,0.639,0.0227,Sell
8,CRDO,17.173,98.52,65.96,6.28,71.05,339.72,24.810,0.0000,Sell
11,UNFI,22.950,23.81,23.96,23.96,25.47,inf,0.890,0.0000,Hold
13,EZPW,17.715,14.19,14.40,23.86,14.98,11.92,1.018,0.0000,Hold
14,ARQT,15.996,15.01,13.32,13.32,14.25,inf,12.540,0.0000,Sell
15,WFC,68.586,82.55,71.83,97.57,74.23,14.85,1.653,0.0200,Sell
17,PYPL,69.126,71.36,67.93,56.37,70.54,16.04,3.426,0.0000,Sell
19,CLS,74.924,161.03,109.44,41.00,116.36,45.11,11.730,0.0000,Sell
20,CCL,17.866,28.66,22.43,21.98,23.46,15.01,3.883,0.0000,Sell
23,ATGE,86.385,117.99,117.11,89.91,123.38,19.96,2.959,0.0000,Hold


In [75]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,WLDN,0.0,73.92,55.96,31.60,58.93,43.74,4.291,0.0000,Sell
1,SSRM,0.0,12.66,11.40,14.82,12.04,30.88,0.810,0.0000,Sell
3,PUK,0.0,24.71,22.53,28.02,23.09,14.71,1.813,0.0185,Sell
4,FINV,0.0,10.60,8.54,20.12,9.05,7.63,1.248,0.0292,Sell
5,PSIX,0.0,72.02,45.67,21.07,50.99,20.40,19.650,0.0000,Sell
6,NGD,0.0,4.70,4.12,2.84,4.35,26.11,3.600,0.0000,Sell
7,GFI,0.0,24.15,22.53,16.58,23.51,17.50,4.156,0.0230,Sell
9,COMM,0.0,7.53,5.55,5.55,5.98,inf,30.360,0.0000,Sell
10,BKTI,0.0,40.96,45.27,28.09,50.03,15.69,4.649,0.0000,Hold
12,MFC,0.0,30.48,30.34,35.74,31.27,15.79,1.580,0.0402,Hold


In [76]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [77]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,LRN,137.913,134.49,135.85,84.35,149.16,21.11,4.130,0.0000,Hold
3,UNFI,22.200,23.81,22.99,22.99,25.47,inf,0.890,0.0000,Hold
5,EAT,100.482,165.09,146.97,37.53,161.70,23.19,28.610,0.0000,Sell
6,EZPW,15.460,14.19,13.82,23.86,14.98,11.92,1.018,0.0000,Hold
7,ARQT,15.932,15.01,12.78,12.78,14.25,inf,12.540,0.0000,Sell
8,WFC,60.671,82.55,68.93,97.57,74.23,14.85,1.653,0.0200,Sell
9,ITRN,35.710,39.35,34.30,29.90,37.26,14.47,4.106,0.0513,Sell
10,CRDO,0.010,98.52,63.30,6.28,71.05,339.72,24.810,0.0000,Sell
11,PYPL,68.738,71.36,65.19,56.37,70.54,16.04,3.426,0.0000,Sell
12,ALL,195.419,193.33,186.48,194.27,200.69,13.21,2.571,0.0199,Hold


In [78]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,WLDN,0.0,73.92,53.70,31.60,58.93,43.74,4.291,0.0000,Sell
1,SSRM,0.0,12.66,10.94,14.82,12.04,30.88,0.810,0.0000,Sell
4,MFC,0.0,30.48,29.12,35.74,31.27,15.79,1.580,0.0402,Hold
14,QTWO,0.0,89.19,81.03,81.03,88.04,inf,10.190,0.0000,Sell
20,SYF,0.0,70.61,53.94,100.52,58.38,9.67,1.750,0.0180,Sell
21,ATGE,0.0,117.99,112.38,89.91,123.38,19.96,2.959,0.0000,Hold
22,BRK-B,0.0,475.86,474.76,624.53,508.02,12.69,1.569,0.0000,Hold
23,SFM,0.0,159.70,152.59,44.92,167.16,35.73,12.130,0.0000,Hold
25,BLBD,0.0,44.95,36.56,25.81,39.85,13.96,7.450,0.0000,Sell
26,GM,0.0,53.39,44.89,128.23,48.55,7.46,0.797,0.0122,Sell


In [79]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'RDWR'}, # Sweden - Communications
              {'symbol':'PGY'}, # Sweden - Communications
              {'symbol':'STX'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [80]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.94,24.00,28.60,24.56,10.10,2.379,0.0535,Hold
1,KARO,0,49.00,48.02,17.87,50.88,29.17,8.840,0.0221,Hold
2,QFIN,0,43.09,40.64,72.91,42.38,6.64,1.804,0.0300,Sell
3,TSM,0,230.40,186.68,93.81,191.25,27.04,7.650,0.0145,Sell
4,CCU,0,12.83,13.85,16.85,14.23,13.51,1.471,0.0353,Hold
5,ABEV,0,2.35,2.41,2.44,2.47,14.69,2.165,0.0634,Hold
6,TIMB,0,18.99,17.69,198.48,18.09,14.95,0.021,0.0768,Sell
7,ASR,0,308.31,311.13,283.93,319.29,12.72,3.179,0.0771,Hold
8,RYAAY,0,57.47,52.49,42.66,53.57,16.85,3.694,0.0162,Sell
9,AMX,0,17.54,16.67,10.63,17.05,35.80,2.607,0.0305,Sell


In [81]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.94,21.90,28.60,23.60,10.10,2.379,0.0535,Sell
1,KARO,0,49.00,43.90,17.87,48.90,29.17,8.840,0.0221,Sell
2,QFIN,0,43.09,38.76,72.91,42.93,6.64,1.804,0.0300,Sell
3,TSM,0,230.40,174.36,93.81,190.39,27.04,7.650,0.0145,Sell
4,CCU,0,12.83,13.05,16.85,14.15,13.51,1.471,0.0353,Hold
5,ABEV,0,2.35,2.09,2.44,2.25,14.69,2.165,0.0634,Sell
6,TIMB,0,18.99,14.76,198.48,15.90,14.95,0.021,0.0768,Sell
7,ASR,0,308.31,279.94,283.93,303.50,12.72,3.179,0.0771,Sell
8,RYAAY,0,57.47,45.97,42.66,49.61,16.85,3.694,0.0162,Sell
9,AMX,0,17.54,14.86,10.63,16.03,35.80,2.607,0.0305,Sell


In [82]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, margin_of_safety=0.95, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [83]:
mag7_test_table

[['MSFT', 0, 503.32, 420.05, 138.91, 449.67, 38.75, 11.62, 0.0067, 'Sell'],
 ['AAPL', 0, 211.16, 190.17, 31.19, 205.33, 33.1, 47.5, 0.0051, 'Sell'],
 ['NVDA', 0, 164.92, 121.6, 19.12, 132.08, 53.2, 47.97, 0.0003, 'Sell'],
 ['AMZN', 0, 225.02, 189.36, 78.33, 204.39, 36.24, 7.81, 0, 'Sell'],
 ['GOOG', 0, 181.31, 157.56, 93.46, 171.01, 20.26, 6.37, 0.0047, 'Sell'],
 ['GOOGL', 0, 180.19, 155.97, 93.48, 169.42, 20.13, 6.33, 0.0048, 'Sell'],
 ['META', 0, 717.51, 587.58, 252.47, 635.55, 28.41, 9.75, 0.0028, 'Sell'],
 ['BRK-B', 0, 475.86, 474.76, 624.53, 508.02, 12.69, 1.569, 0, 'Hold'],
 ['TSLA', 0, 313.51, 282.55, 37.4, 313.63, 178.13, 13.53, 0, 'Hold']]

In [84]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,503.32,420.05,138.91,449.67,38.75,11.620,0.0067,Sell
1,AAPL,0,211.16,190.17,31.19,205.33,33.10,47.500,0.0051,Sell
2,NVDA,0,164.92,121.60,19.12,132.08,53.20,47.970,0.0003,Sell
3,AMZN,0,225.02,189.36,78.33,204.39,36.24,7.810,0.0000,Sell
4,GOOG,0,181.31,157.56,93.46,171.01,20.26,6.370,0.0047,Sell
5,GOOGL,0,180.19,155.97,93.48,169.42,20.13,6.330,0.0048,Sell
6,META,0,717.51,587.58,252.47,635.55,28.41,9.750,0.0028,Sell
7,BRK-B,0,475.86,474.76,624.53,508.02,12.69,1.569,0.0000,Hold
8,TSLA,0,313.51,282.55,37.40,313.63,178.13,13.530,0.0000,Hold


In [85]:
# Machine Learning

In [86]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [87]:
# Feature selection and normalization

In [88]:
# Build model

In [89]:
# Train model

In [90]:
# Evaluate model

In [91]:
# Refine model through hyperparameter tuning